# 02: Train Price Models

Train XGBoost and LightGBM specialist models.

In [ ]:
# Setup
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path().absolute().parent.parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from models import XGBoostModel, LightGBMModel
from utils.config import PROCESSED_DATA_DIR, SPECIALIST_MODELS_DIR

TICKER = "AAPL"

In [ ]:
# Load features
features = pd.read_csv(PROCESSED_DATA_DIR / f"{TICKER}_features.csv", index_col=0, parse_dates=True)

# Prepare data
feature_cols = [col for col in features.columns 
                if col not in ['target_return_1d', 'target_direction', 'open', 'high', 'low', 'close', 'volume']]
X = features[feature_cols].fillna(0)
y = features['target_direction']

# Remove rows with zero target (no clear direction)
mask = y != 0
X = X[mask]
y = y[mask]

print(f"Training data: {len(X)} samples, {len(feature_cols)} features")

In [ ]:
# Train XGBoost
xgb_model = XGBoostModel(min_confidence=0.6)
xgb_metrics = xgb_model.train(X, y)
print(f"XGBoost metrics: {xgb_metrics}")

# Save
xgb_model.save(SPECIALIST_MODELS_DIR / f"{TICKER}_xgb.model")
print(f"✓ Saved XGBoost model")

In [ ]:
# Train LightGBM
lgb_model = LightGBMModel(min_confidence=0.6)
lgb_metrics = lgb_model.train(X, y)
print(f"LightGBM metrics: {lgb_metrics}")

# Save
lgb_model.save(SPECIALIST_MODELS_DIR / f"{TICKER}_lgb.txt")
print(f"✓ Saved LightGBM model")